In [1]:
#!pip install neuralforecast

  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached rpds_py-0.18.0-cp311-none-win_amd64.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/213.6 kB ? eta -:--:--
   ----------- ---------------------------- 61.4/213.6 kB 3.2 MB/s eta 0:00:01
   -------------------------------------- - 204.8/213.6 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 213.6/213.6 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/86.3 kB ? eta -:--:--
   ---------------------------------------- 86.3/86.3 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/801.9 kB ? eta -:--:--
   ------------ --------------------------- 256.0/801.9 kB 5.2 MB/s eta 0:00:01
   ---------------------- ----------------- 450.6/801.9 kB 4.7 MB/s eta 0:00:01
   ------------------

In [3]:
import pandas as pd

In [20]:
data = pd.read_excel("C:/Users/Usuario/Documents/Projectes/ChallengeAuditoria/challenge-auditoria/data/inventory_data_month.xlsx")


ts = ['Jan_2022', "Feb_2022", "Mar_2022", "Apr_2022", "May_2022", "Jun_2022", "Jul_2022", "Aug_2022", "Sep_2022", "Oct_2022", "Nov_2022", "Dec_2022", 'Jan_2023', "Feb_2023", "Mar_2023", "Apr_2023", "May_2023", "Jun_2023", "Jul_2023", "Aug_2023", "Sep_2023", "Oct_2023", "Nov_2023", "Dec_2023"]
data.fillna(0, inplace=True)
print(data.columns)
ts.extend(["material", "vendes_2022", "vendes_2023"])
data = data[ts]

Index(['Unnamed: 0.1', 'Unnamed: 0', 'material', 'unitats_2022', 'vendes_2022',
       'preu_venda_unitari_2022', 'unitats_2023', 'vendes_2023',
       'preu_venda_unitari_2023', 'variacio_preu_venda_unitari_2022_2023',
       'proporcio_variacio_preu_venda_unitari_2022_2023',
       'data_darrera_entrada', 'dies_ultima_entrada', 'data_darrera_sortida',
       'dies_ultima_sortida', 'diferencia_entrada_sortida', 'stock_final_2023',
       'valor_total_stock_2023', 'cost_unitari_stock_2023', 'Jan_2022',
       'Jan_2023', 'Feb_2022', 'Feb_2023', 'Mar_2022', 'Mar_2023', 'Apr_2022',
       'Apr_2023', 'May_2022', 'May_2023', 'Jun_2022', 'Jun_2023', 'Jul_2022',
       'Jul_2023', 'Aug_2022', 'Aug_2023', 'Sep_2022', 'Sep_2023', 'Oct_2022',
       'Oct_2023', 'Nov_2022', 'Nov_2023', 'Dec_2022', 'Dec_2023'],
      dtype='object')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_29640\649198885.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  data.fillna(0, inplace=True)


In [55]:
df = pd.DataFrame(data)

# Unpivot the DataFrame
df = df.melt(id_vars=["material", "vendes_2022", "vendes_2023"], var_name='ds', value_name='y')

# Extract year and month from 'ds' column
df['ds'] = pd.to_datetime(df['ds'], format='%b_%Y')

# Rename the 'material' column to 'unique_id'
df.rename(columns={'material': 'unique_id'}, inplace=True)

print(df)

      unique_id  vendes_2022  vendes_2023         ds             y
0     114230822     54892.49     58951.87 2022-01-01   4876.443011
1     115030252      6825.62      2127.68 2022-01-01      0.000000
2     115070602    100307.84     34522.45 2022-01-01   6065.864778
3     115130209      5179.48      3770.68 2022-01-01    505.955967
4     115170581     26188.50     74195.50 2022-01-01    859.708957
...         ...          ...          ...        ...           ...
9187  182080257     27030.19     17886.40 2023-12-01   1644.182569
9188  182080258      3945.01         0.00 2023-12-01      0.000000
9189  182080259    112860.80    326272.67 2023-12-01  13822.378008
9190  182080260      8513.28     11016.96 2023-12-01      0.000000
9191  182130161    457202.60    390388.11 2023-12-01  30437.723719

[9192 rows x 5 columns]


In [56]:
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS, RNN, NBEATS, TimesNet

nf = NeuralForecast(
    models = [NBEATS(input_size=12, h=12, max_steps=100, hist_exog_list = ["vendes_2022", "vendes_2023"]),
              TimesNet(input_size=12, h=12, max_steps=100)
              ],
    freq = 'M'
)

nf.fit(df=df)
new_df = nf.predict()

c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\utilities\parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
Seed set to 1
Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 2.4 M 
-----------------------------------------------
2.4 M     Trainable params
600       Non-trainable params
2.4 M     Total params
9.690     Total estimated model params size (MB)


c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n

Epoch 8:  33%|███▎      | 4/12 [00:00<00:01,  6.87it/s, v_num=6, train_loss_step=1.47e+4, train_loss_epoch=1.08e+4] 

`Trainer.fit` stopped: `max_steps=100` reached.


Epoch 8:  33%|███▎      | 4/12 [00:00<00:01,  6.78it/s, v_num=6, train_loss_step=1.47e+4, train_loss_epoch=1.08e+4]

...


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 112.05it/s]

GPU available: False, used: False


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 12/12 [00:04<00:00,  2.98it/s]


c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\neuralforecast\core.py:184: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [57]:
new_df.head()

,ds,NBEATS,TimesNet
unique_id,,,
114230822,2023-12-31,3633.244385,4317.026855
114230822,2024-01-31,3702.182129,4136.087891
114230822,2024-02-29,3970.479248,3846.902344
114230822,2024-03-31,4522.752930,4480.974121
114230822,2024-04-30,4360.987305,3594.966309


In [62]:
df = pd.DataFrame(new_df)

# Convert 'ds' column to datetime
df['ds'] = pd.to_datetime(df['ds'])

# Pivot the DataFrame
df_pivot = df.pivot_table(index='unique_id', columns='ds', values='NBEATS', aggfunc='first')

# Reset index to make 'unique_id' a column again
df_pivot.reset_index(inplace=True)

# Rename columns
df_pivot.columns.name = None  # Remove the name of the columns index
df_pivot.columns = ['unique_id'] + [col.strftime('%b_%Y') for col in df_pivot.columns[1:]]

print(df_pivot)

df_pivot = df_pivot.drop(["Dec_2023"], axis=1)
forecasts = df_pivot.set_index('unique_id').T.to_dict(orient='list')
import json
new_forecasts = {key: list(value) for key, value in forecasts.items()}
# Saving the dictionary to a JSON file
with open("../forecast/nbeats.json", "w") as json_file:
    json.dump(new_forecasts, json_file)

     unique_id      Dec_2023      Jan_2024      Feb_2024      Mar_2024  \
0    114230822   3633.244385   3702.182129   3970.479248   4522.752930   
1    115030252    241.708588    153.883667    128.897125    223.399521   
2    115070602   2429.710205   2663.228027   2212.334229   2456.969238   
3    115130209    186.998856    186.759201    189.941040    137.806213   
4    115170581   6678.361328   5253.115723   7045.935547   4255.138184   
..         ...           ...           ...           ...           ...   
378  182080257   1213.146973   1272.343872   1107.849609    968.502869   
379  182080258      0.510572      0.439281      0.414640      0.494350   
380  182080259  22909.841797  19510.150391  14872.630859  25970.746094   
381  182080260    308.198792    516.171265    208.395172    204.455322   
382  182130161  27835.687500  22420.244141  26057.037109  36698.496094   

         Apr_2024      May_2024      Jun_2024      Jul_2024      Aug_2024  \
0     4360.987305   3077.870361   

In [61]:
df = pd.DataFrame(new_df)

# Convert 'ds' column to datetime
df['ds'] = pd.to_datetime(df['ds'])

# Pivot the DataFrame
df_pivot = df.pivot_table(index='unique_id', columns='ds', values='TimesNet', aggfunc='first')

# Reset index to make 'unique_id' a column again
df_pivot.reset_index(inplace=True)

# Rename columns
df_pivot.columns.name = None  # Remove the name of the columns index
df_pivot.columns = ['unique_id'] + [col.strftime('%b_%Y') for col in df_pivot.columns[1:]]

print(df_pivot)

df_pivot = df_pivot.drop(["Dec_2023"], axis=1)
forecasts = df_pivot.set_index('unique_id').T.to_dict(orient='list')
import json
new_forecasts = {key: list(value) for key, value in forecasts.items()}
# Saving the dictionary to a JSON file
with open("../forecast/timesnet.json", "w") as json_file:
    json.dump(new_forecasts, json_file)

     unique_id      Dec_2023      Jan_2024      Feb_2024      Mar_2024  \
0    114230822   4317.026855   4136.087891   3846.902344   4480.974121   
1    115030252    141.553345    141.578888    183.549026    173.906799   
2    115070602   2150.878906   2160.991943   2329.116455   2347.580566   
3    115130209    262.094116    174.616577    292.652283    289.355011   
4    115170581   4915.030273   4434.943848   6029.007812   4628.212891   
..         ...           ...           ...           ...           ...   
378  182080257   1203.366699   1141.923340   1270.328003   1361.576416   
379  182080258      0.542615      0.415263      0.503743      0.907837   
380  182080259  24570.617188  20779.468750  24634.878906  24696.648438   
381  182080260    670.174438    -40.097412    195.522522    603.965332   
382  182130161  28616.023438  23385.347656  29289.705078  33768.546875   

         Apr_2024      May_2024      Jun_2024      Jul_2024      Aug_2024  \
0     3594.966309   3789.168701   

# Model preentrenat

No furula

In [ ]:
from neuralforecast.models import LSTM, NHITS, RNN, NBEATS

nf2 = NBEATS.load_from_checkpoint(checkpoint_path='./nbeats_m4.ckpt',h=12,input_size=12)
Y_hat_df = nf2.predict().reset_index()
Y_hat_df.head()